In [19]:
import re
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.corpus import wordnet
import io
import os
import glob
import itertools
import pandas as pd
from sklearn.utils import resample

In [20]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [21]:
path1 = '2013'

In [22]:
def read_all_files(path):
    criteria = os.listdir(path)
    if len (criteria) == 0:
        raise Exception ('This is an empty file')
    else:
        files = glob.glob(path + '/*')
        frame = []
        for f in files:
            text = open(f, 'r', encoding = 'latin-1').read()
            frame.append(text)
        return frame

In [23]:
c2013 = read_all_files(path1)
print(len(c2013))

730


In [24]:
#check if Capital Name is stopword

stop_words = stopwords.words('english')

def check_stopwords(word):
    if word.lower() not in stop_words:
        return True
    else:
        return False


In [25]:
#sentence tokenize

sentence13 = []
for item in c2013:
    sentence13.extend(sent_tokenize(item))
    
print(len(sentence13))

683290


In [26]:
def sent_token(corpus):
    sen_corpus = []
    for file in corpus:
        token_file = sent_tokenize(file)
        sen_corpus.append(token_file)
    return sen_corpus

In [27]:
sen_corpus = sent_token(c2013)
print(len(sen_corpus))

730


In [28]:
#download our train data
percent_path = 'all/percentage.csv'

In [29]:
def process_percentage(name):
    
    
    name = name.strip().split(',')
    name = filter(None, name)
    name = list(map(lambda x: x.strip(),name))
    
    return name
    

In [30]:
percent_file = open(percent_path, 'r', encoding = 'latin-1').readlines()
percent_names = map(lambda x: process_percent(x), percent_file)
percent_names = list(set(percent_names))

In [31]:
split0 = int(0.6* len(sen_corpus))
split1 = int(0.8* len(sen_corpus))

training_set = sen_corpus[0:split0]
validation_set = sen_corpus[split0:split1]
testing_set = sen_corpus[split1:]

assert len(sen_corpus) == len(training_set) + len(validation_set) + len(testing_set)

In [32]:
def find_names(article):
    
    rep = '% | percentage| percent'
    pattern = re.compile(rep)
    
    names = map(lambda x: re.findall(pattern,x ), article)
    names = filter(None, names)
    names = itertools.chain(*names)
    
    names = filter(check_stopwords, names)
    
    names = set(names)
    
    return names    

In [33]:
ceo = set(ceo_names)

In [34]:
def get_feature(article, names):
    
    def check_in_sentence(name, sentence):
        return ((' '+ name +' ' in sentence) or (name + ' ' == sentence[:len(name)+1] or (' ' + name == sentence[-len(name)+1:])))
 
    vectors = []
    features = {}
    for name in names:
        sentences = filter(lambda x: check_in_sentence(name,x), article)
        
        for sentence in sentences:
            
            features['is_percent'] = int(name in percent)
            
            s = word_tokenize(sentence)

            features['index_first'] = first_index
            first_index = s.index(first)
            
            word_list1 = nltk.pos_tag(name[first_index - 1])
            word_list2 = nltk.pos_tag(name[first_index + word_length + 1])
            
            s = word_tokenize(sentence)
            
            features['front_pos'] = word_list1[1]
            features['last_pos'] = word_list2[1]
            if sentence.count('percent') > 0 or sentence.count('%') > 0:
                features['specific'] = 1
            else:
                features['specific'] = 0
            
            vectors.append(features)
    return vectors  

In [35]:
train_data = []
for i, article in enumerate(training_set):
    names = find_names(article)
    vectors = get_feature(article, names)
    train_data.extend(vectors)

    
df = pd.DataFrame(train_data)

AttributeError: 'tuple' object has no attribute 'lower'

In [ ]:
df['is_ceo'].value_counts()

In [ ]:
df_positive = df[df.is_ceo == 1]
df_negative = df[df.is_ceo == 0]

n_per_class = 10*len(df_positive)

df_positive = resample(df_positive, replace = True, n_samples = n_per_class)
df_negative = resample(df_negative, replace = False, n_samples = n_per_class)
df1 = pd.concat([df_positive, df_negative])

In [ ]:
df1

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(n_estimators = 20)
col = [c for c in df1.columns if c not in ('is_ceo','name')]
rfc.fit(df1[col], df1['is_ceo'])